In [34]:
from datasets import load_dataset
import pandas as pd
import numpy as np
from os import listdir
from IPython.display import display, HTML
import json
import re


In [70]:
test = load_dataset("xsum", split='test')
#switch to raw maynes file if maynez_docset doesn't exist yet, use method below to generate
df = pd.read_csv('./maynez_docset.csv')

Using custom data configuration default
Reusing dataset xsum (/home/prasann/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)


In [4]:
# matchse maynez examples to corresponding xsum documents
def load_maynez_docset():
    doclist = []
    sumlist = []
    indlist = []
    for index, row in df.iterrows():
        t = test['id'].index(str(row['bbcid']))
        if t>=0:
            doclist.append(test[t]['document'])
            sumlist.append(test[t]['summary']);
            indlist.append(t)
    df['doc'] = doclist
    df['corr'] = sumlist
    df['dataind'] = indlist
    df.to_csv('maynez_docset.csv', index=False)
    return df

In [10]:
#TODO insert this into lattice gen code, make dataset subset based on this array
test.select(df['dataind'].unique())

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 498
})

In [45]:
df.loc[df.is_factual=='yes']

,bbcid,system,summary,is_factual,worker_id,doc,corr,dataind
9,27838706,BERTS2S,aberdeen have appointed arbroath boss paul she...,yes,wid_0,"Former Dons midfielder Sheerin, 39, has been p...",Paul Sheerin has left his post as Arbroath man...,7268
10,27838706,BERTS2S,aberdeen have appointed arbroath boss paul she...,yes,wid_1,"Former Dons midfielder Sheerin, 39, has been p...",Paul Sheerin has left his post as Arbroath man...,7268
21,31566848,BERTS2S,a woman was assaulted and assaulted in a mcdon...,yes,wid_0,The victim was queuing for food at the branch ...,"A woman had her hair set alight in a ""nasty an...",2915
22,31566848,BERTS2S,a woman was assaulted and assaulted in a mcdon...,yes,wid_1,The victim was queuing for food at the branch ...,"A woman had her hair set alight in a ""nasty an...",2915
23,31566848,BERTS2S,a woman was assaulted and assaulted in a mcdon...,yes,wid_2,The victim was queuing for food at the branch ...,"A woman had her hair set alight in a ""nasty an...",2915
...,...,...,...,...,...,...,...,...
5444,36086291,PtGen,prince william has said she is ``incredibly im...,yes,wid_0,He was speaking in a BBC interview in which he...,Prince William has answered criticism of his c...,1799
5445,36086291,PtGen,prince william has said she is ``incredibly im...,yes,wid_1,He was speaking in a BBC interview in which he...,Prince William has answered criticism of his c...,1799
5446,36086291,PtGen,prince william has said she is ``incredibly im...,yes,wid_2,He was speaking in a BBC interview in which he...,Prince William has answered criticism of his c...,1799
5528,37895159,PtGen,bournemouth manager eddie howe has urged his p...,yes,wid_0,The Cherries went down 2-1 at Sunderland on Sa...,Bournemouth boss Eddie Howe admits his side's ...,7810


2022-03-25 02:35:11.820359: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-25 02:35:11.820401: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


AssertionError: Dataset format must be numpy before exporting

In [123]:
genpath = "./output/text/"
vispath = "./output/html/"
out = "sum_xsum_astar_16_15_False_0.4_True_False_4_5_zip_0.75_0.0_0.9/"
allfiles = listdir(genpath+out)
filterid = ".*(?:\D|^)(\d+)"

def filter_json(afs):
    result = []
    for a in afs:
        if 'json' in a:
            result.append(a)
    return result

def get_top (genlist):
    maxg = -1000
    best = []
    for g in genlist:
        if g[2]['rouge2']> maxg:
            maxg = g[2]['rouge2']
            best = g
    return best

def compare_maynez (ind):
    files = filter_json(allfiles)
    # read file
    with open(genpath+out+files[ind], 'r') as myfile:
        data=myfile.read()

    # parse file
    obj = json.loads(data)
    #display(HTML(data))
    xsumid = (re.search(filterid, files[ind])).group(1)
    print("ID: "+str(xsumid))
    
    print("Top1: ")
    print(get_top(obj['sample']))
    a  = df.loc[df.bbcid==int(xsumid)]
    maynez_false = a.loc[a['is_factual'].str.contains("no")]
    maynez_yes = a.loc[a['is_factual'].str.contains("yes")]
    print("Correct Summary:\n"+maynez_false.iloc[0]['corr']+"\n")
    print ("Maynez summaries (non-factual) ")
    mf = maynez_false['summary'].unique()
    my = maynez_yes['summary'].unique()
    for m in range(0, len(mf)):
        print("Summ "+str(m)+":"+mf[m])

    print ("Maynez summaries (factual) ")
    for m in range(0, len(my)):
        print("Summ "+str(m)+":"+my[m])
    print("\n\n")

    

In [125]:
for i in range(0, 21):
    compare_maynez(i)

ID: 35977985
Top1: 
['Braintree manager Keith Cowley has hailed his team\'s performance after their 1-0 defeat by Barrow is their "toughest challenge to date" despite the defeat.', 37, {'rouge2': 0.13636363636363635}]
Correct Summary:
Braintree manager Danny Cowley said the task ahead of his team is "a brilliant challenge" after the defeat at Barrow.

Maynez summaries (non-factual) 
Summ 0:braintree boss steve cowley says his side's 1-0 victory over barrow in the national league was " remarkable ".
Summ 1:cowdenbeath head coach shaun barrow says his side are in a " tough " way to claim their first national league win of the season.
Summ 2:braintree town manager gary cowley says his side have a ``good chance''of winning the national league play-offs.
Summ 3:braintree manager nicola cowley has urged his players to be ``tough''after winning the premier league title.
Maynez summaries (factual) 
Summ 0:braintree boss steve cowley says his side's 1-0 victory over barrow in the national leagu

In [28]:
obj['sample'][0]

["As David Cameron prepares for the first Commons vote on the UK's membership of the European Union, what do its MPs think about the issue?",
 30,
 {'rouge2': 0.07692307692307691}]

In [86]:
a  = df.loc[df.bbcid==int('39979364')]
b = a.loc[a['is_factual'].str.contains("no")]
b

,bbcid,system,summary,is_factual,worker_id,doc,corr,dataind
60,39979364,BERTS2S,the london 2012 para-athletics world champions...,no,wid_0,"Symbols of every event feature on the design, ...",The World Championships medals that will be wo...,3603
61,39979364,BERTS2S,the london 2012 para-athletics world champions...,no,wid_1,"Symbols of every event feature on the design, ...",The World Championships medals that will be wo...,3603
62,39979364,BERTS2S,the london 2012 para-athletics world champions...,no,wid_2,"Symbols of every event feature on the design, ...",The World Championships medals that will be wo...,3603
4291,39979364,PtGen,the world para athletics championships are tak...,no,wid_0,"Symbols of every event feature on the design, ...",The World Championships medals that will be wo...,3603
4292,39979364,PtGen,the world para athletics championships are tak...,no,wid_1,"Symbols of every event feature on the design, ...",The World Championships medals that will be wo...,3603
4293,39979364,PtGen,the world para athletics championships are tak...,no,wid_2,"Symbols of every event feature on the design, ...",The World Championships medals that will be wo...,3603
